In [1]:
!pip install langchain
!pip install unstructured
!pip install langchain-google-genai
!pip install Cython
!pip install tiktoken

In [2]:
!pip install langchain-astradb

In [3]:
!pip install datasets pypdf

In [25]:
!pip install pdf2image
!pip install pdfminer.six
!pip install unstructured[pdf]
!pip install "unstructured[pptx]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 16.1 MB/s eta 0:00:00


In [16]:
!sudo apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 29 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.6 [186 kB]
Fetched 186 kB in 0s (744 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package poppler-utils.
(Reading database ... 124947 

In [17]:
!sudo apt-get install libleptonica-dev tesseract-ocr libtesseract-dev python3-pil tesseract-ocr-eng tesseract-ocr-script-latn

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libarchive-dev libimagequant0 libraqm0 python3-olefile tesseract-ocr-osd
Suggested packages:
  python-pil-doc
The following NEW packages will be installed:
  libarchive-dev libimagequant0 libleptonica-dev libraqm0 libtesseract-dev
  python3-olefile python3-pil tesseract-ocr tesseract-ocr-eng
  tesseract-ocr-osd tesseract-ocr-script-latn
0 upgraded, 11 newly installed, 0 to remove and 29 not upgraded.
Need to get 39.9 MB of archives.
After this operation, 123 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libarchive-dev amd64 3.6.0-1ubuntu1.3 [581 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libimagequant0 amd64 2.17.0-1 [34.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libleptonica-dev amd64 1.82.0-3build1 [1,562 kB]
Get:4 http://archive.ubuntu

In [18]:
!pip install unstructured-pytesseract
!pip install tesseract-ocr

  Preparing metadata (setup.py) ... done
  Created wheel for tesseract-ocr: filename=tesseract_ocr-0.0.1-cp311-cp311-linux_x86_64.whl size=179100 sha256=588c20474bf436e2aaf3858089c3380f922c0954312c3f5bb07806f1facaace2
  Stored in directory: /root/.cache/pip/wheels/90/83/3c/d2b68d844d169d6015fc2ad8c93207d778829c87e26c6f2206
Successfully built tesseract-ocr


In [5]:
import os
from getpass import getpass

from datasets import (
    load_dataset,
)

from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.indexes import VectorstoreIndexCreator

In [6]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [7]:
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

## Using Unstructured for loading Multiple Pdfs

In [20]:
from langchain_community.document_loaders import DirectoryLoader

In [9]:
root_dir="/content/"

In [10]:
pdf_folder_path = f'{root_dir}/docs/'

In [11]:
os.listdir(pdf_folder_path)

['M_RAG.pdf', 'Potential-of-GenAI-in-2024.pptx']

In [22]:
loader = DirectoryLoader('/content/docs')

In [23]:
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=64)

In [26]:
docs = loader.load_and_split(text_splitter=splitter)

In [27]:
len(docs)

132

## Database Connect

In [28]:
from langchain_astradb import AstraDBVectorStore
from langchain.indexes import VectorstoreIndexCreator

In [29]:
ASTRA_DB_API_ENDPOINT="https://72385aec-0c35-4d49-9f54-b03bdf62da09-us-east-2.apps.astra.datastax.com"
ASTRA_DB_APPLICATION_TOKEN="AstraCS:FohmtZXIWlZUXXFZdqnQHlOO:139ee86917037cc80edf934eb5dfbcdf3494312128ea963d52c66a3e6fcf00a2"
ASTRA_DB_KEYSPACE="default_keyspace"

In [30]:
vstore = AstraDBVectorStore(
    embedding = embeddings,
    collection_name = "multidoc_vector",
    api_endpoint = ASTRA_DB_API_ENDPOINT,
    token = ASTRA_DB_APPLICATION_TOKEN,
    namespace = ASTRA_DB_KEYSPACE,
)

## Ingestion

In [31]:
inserted_ids = vstore.add_documents(docs)

In [32]:
print(f"\nInserted {len(inserted_ids)} documents.")


Inserted 132 documents.


## Prompt

In [35]:
prompt_template = """
You are an AI philosopher drawing insights from the impact of "rag," and "genai."
Craft thoughtful answers based on these, mixing and matching existing paths.
Your responses should be concise and strictly related to the provided context.

ROADMAP CONTEXT:
{context}

QUESTION: {question}

YOUR ANSWER:"""

In [36]:
prompt_template = ChatPromptTemplate.from_template(prompt_template)

## Retrieve

In [37]:
retriever = vstore.as_retriever(search_kwargs={"k": 3})

In [38]:
retriever

VectorStoreRetriever(tags=['AstraDBVectorStore', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_astradb.vectorstores.AstraDBVectorStore object at 0x7c2034afd4d0>, search_kwargs={'k': 3})

## Model Selection

In [44]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

### Creating Chain

In [45]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [47]:
chain.invoke("Can you tell about generative ai?")

'Generative AI, employed in M-RAG architectures, utilizes large language models (LLMs) for text generation.  Retrieval augmented generation (RAG) improves LLM results by revising retrieved memory through reranking, eliminating irrelevant content, summarizing, or compressing prompts before feeding back to the LLM.  M-RAG specifically uses a retrieval-then-generation process with agents focusing on retrieval and generation aspects.'